In [6]:
import pandas as pd
import numpy as np

In [15]:
gym_data = pd.read_csv("gym/gym_exercise_dataset.csv")
gym_data.head()

,Exercise Name,Equipment,Variation,Utility,Mechanics,Force,Preparation,Execution,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Main_muscle,Difficulty (1-5),Secondary Muscles,parent_id
0,Neck Flexion,Cable,No,Basic or Auxiliary,Isolated,Pull,Sit on bench facing away from middle pulley. P...,Move head away from pulley by bending neck for...,"Sternocleidomastoid,","None,","Rectus Abdominis, Obliques,",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
1,Neck Flexion,Lever (plate loaded),No,Basic or Auxiliary,Isolated,Pull,Sit on seat in machine. Position padded lever ...,Move head forward by flexing neck until chin t...,"Sternocleidomastoid,","None,","Latissimus Dorsi, Deltoid, Posterior, Rhomboid...",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
2,Lateral Neck Flexion,Lever (plate loaded),No,Auxiliary,Isolated,Pull,Sit on seat in machine with feet apart . Pos...,Move head down to side by laterally flexing ne...,"Sternocleidomastoid,","Splenius, Erector Spinae, Levator Scapulae, Tr...","Latissimus Dorsi, Pectoralis Major, Sternal, P...",NaN,NaN,Neck,2,"Sternocleidomastoid, Levator Scapulae",NaN
3,Neck Flexion,Lever (selectorized),No,Basic or Auxiliary,Isolated,Pull,Sit on seat in machine. Position padded lever ...,Move head forward by flexing neck until chin t...,"Sternocleidomastoid,","None,","Latissimus Dorsi, Deltoid, Posterior, Rhomboid...",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
4,Lateral Neck Flexion,Lever (selectorized),No,Auxiliary,Isolated,Pull,Sit on seat in machine with feet apart. Positi...,Move head down to side by laterally flexing ne...,"Sternocleidomastoid,","Splenius, Erector Spinae, Levator Scapulae, Tr...","Latissimus Dorsi, Pectoralis Major, Sternal, P...",NaN,NaN,Neck,2,"Sternocleidomastoid, Levator Scapulae",NaN


In [8]:
exe_data = pd.read_csv("gym/exercises.csv")
# exe_data.head()

In [9]:

exe_data.columns

Index(['name', 'force', 'level', 'mechanic', 'equipment', 'primaryMuscles',
       'secondaryMuscles', 'instructions', 'category', 'images', 'id'],
      dtype='object')

In [10]:
# df= exe_data['target'].unique()
# df

In [11]:
exe_data.dtypes

name                object
force               object
level               object
mechanic            object
equipment           object
primaryMuscles      object
secondaryMuscles    object
instructions        object
category            object
images              object
id                  object
dtype: object

In [12]:
gym_data.columns

Index(['Exercise Name', 'Equipment', 'Variation', 'Utility', 'Mechanics',
       'Force', 'Preparation', 'Execution', 'Target_Muscles',
       'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles',
       'Dynamic_Stabilizer_Muscles', 'Main_muscle', 'Difficulty (1-5)',
       'Secondary Muscles', 'parent_id'],
      dtype='object')

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
# Assuming the dataset is a CSV
df = pd.read_csv('gym/exercises.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
                       'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# Target variable (target muscle)
y = df['target']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')
# Example: Recommend exercises similar to a specific target
def recommend_exercises(target_input, k=5):
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Get the predicted probabilities for the input
    probabilities = rf.predict_proba(X_test)
    
    # # Get indices of top k exercises similar to the input
    recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
    # # Decode the recommended target exercises
    recommendations = df.iloc[recommended_indices]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'triceps'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)





KeyError: 'bodyPart'

In [ ]:
# # KNN model
# knn = KNeighborsClassifier(n_neighbors=5)

# # Train the model
# knn.fit(X_train, y_train)

# # Predict on test set
# y_pred_knn = knn.predict(X_test)

# # Evaluate accuracy
# accuracy_knn = accuracy_score(y_test, y_pred_knn)
# print(f'KNN Accuracy: {accuracy_knn * 100:.2f}%')



In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib  # For saving and loading the model
import os

# Load the dataset (update the path as needed)
df = pd.read_csv('gym/exercises.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
                       'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# Target variable (target muscle)
y = df['target']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')

# Save the trained model as a .pkl file
model_filename = 'exercise_recommender_model.pkl'
joblib.dump(rf, model_filename)
print(f'Model saved as {model_filename}')

# Save the label encoders for future use
encoder_filename = 'label_encoders.pkl'
joblib.dump(label_encoders, encoder_filename)
print(f'Label encoders saved as {encoder_filename}')

# Function to recommend exercises similar to a specific target
def recommend_exercises(target_input, k=5):
    # Error handling for invalid inputs
    if target_input not in label_encoders['target'].classes_:
        return f"Invalid target: {target_input}. Available options: {label_encoders['target'].classes_}"
    
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Get the predicted probabilities for all test data
    probabilities = rf.predict_proba(X_test)
    
    # Get indices of top k exercises similar to the input
    recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
    # Decode the recommended target exercises
    recommendations = df.iloc[recommended_indices]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'quads'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)


KeyError: 'bodyPart'

In [ ]:
import joblib
# # Save the RandomForest model and label encoders
joblib.dump(rf, 'exe_model.pkl')
joblib.dump(label_encoders, 'exe_label_encoders.pkl')

['exe_label_encoders.pkl']

In [ ]:
# # Example: Recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Find the k-nearest neighbors
#     distances, indices = knn.kneighbors(X[X['target'] == target_encoded], n_neighbors=k)
    
#     # Get the recommended exercises
#     recommended_exercises = df.iloc[indices[0]]
#     return recommended_exercises[['name', 'target', 'bodyPart', 'equipment']]

# # Recommend exercises similar to 'biceps'
# recommend_exercises('biceps', k=5)


In [ ]:
# # Feature importance from the random forest
# importances = rf.feature_importances_
# feature_names = X.columns

# # Display the feature importance
# for name, importance in zip(feature_names, importances):
#     print(f'{name}: {importance:.4f}')


In [ ]:
# import pickle
# # Save the Random Forest model and label encoders
# with open('exercise_model.pkl', 'wb') as file:
#     pickle.dump({'model': rf, 'label_encoders': label_encoders}, file)